In [1]:
import pandas as pd 
import numpy as np 
import h5py, sys, traceback, os, tqdm, time
from script.utilize import pdgid, purity_classifier
import multiprocessing as mp

In [2]:
NN_RESULT = "./data/results_2020_10_17_nofilter.npz"

results = np.load(NN_RESULT)
full_dataset = results["dataset"]
full_predictions = results["predictions"]
full_targets = results["targets"]
n_match = results["n_match"]
num_samples = full_predictions.shape[0]

num_jets = (full_dataset.sum(-1) > 0).sum(-1)

In [3]:
chi2_storage = []
delta_R_storage = []
NUM_CHI2 = 59

for i in tqdm.trange(NUM_CHI2):
    with h5py.File(f"./backup/evaluate_new_with_JES/output_vector_{i + 1}.h5", 'r') as file:
        chi2_result = np.stack(file["Chi2_result"][:]).astype(np.int)
        delta_R_result = np.stack(file["Delta_R_result"][:]).astype(np.int)
    if i == 0:
        with h5py.File(f"./backup/evaluate_new_with_JES/event_recod_testing_with_chi2_new_{i + 1}.h5", 'r') as file:
            tmp_jet_pt = file["jet_pt"][:]
            N_match_top_in_event = file['N_match_top_in_event'][:]
    else: 
        with h5py.File(f"./backup/evaluate_new_with_JES/event_recod_testing_with_chi2_new_{i + 1}.h5", 'r') as file:
            tmp_jet_pt = np.concatenate((tmp_jet_pt, file["jet_pt"][:]))
            N_match_top_in_event = np.concatenate((N_match_top_in_event, file["N_match_top_in_event"][:]))
    chi2_storage.append(chi2_result)
    delta_R_storage.append(delta_R_result)

chi2_result = np.concatenate(chi2_storage)
delta_R_result = np.concatenate(delta_R_storage)
jet_pt = tmp_jet_pt
del chi2_storage, delta_R_storage

100%|██████████| 59/59 [00:08<00:00,  7.32it/s]


In [4]:
NN_n_match_2_idx = np.where(n_match == 2)
NN_n_match_1_idx = np.where(n_match == 1)

NN_six_jet_idx = np.where(num_jets == 6)
NN_seven_jet_idx = np.where(num_jets == 7)
NN_eight_jet_idx = np.where(num_jets >= 8)

In [5]:
num_jet = np.zeros(len(jet_pt))
for i in range(len(jet_pt)):
    num_jet[i] = len(jet_pt[i])

chi2_six_jet = np.where(num_jet == 6)
chi2_seven_jet = np.where(num_jet == 7)
chi2_eight_jet = np.where(num_jet >= 8)

chi2_n_match_2_idx = np.where(N_match_top_in_event == 2)
chi2_n_match_1_idx = np.where(N_match_top_in_event == 1)

In [6]:
def compute(target, source):
    MODEL = 'ttbar'
    correct_left = np.full(len(target), 99999)
    correct_right = np.full(len(target), 99999)
    case = np.zeros(len(target))

    for i in tqdm.trange(len(target)):

        left_target = target[i][:3]
        right_target = target[i][3:]

        left_src = source[i][:3]    
        right_src = source[i][3:]  

        if np.sum(left_src < 0 ) >0:
            correct_left[i] = 4       
        if np.sum(right_src < 0) >0:
            correct_right[i] = 4

        if correct_left[i] != 4 and correct_right[i] != 4:
            correct_left[i], correct_right[i] = purity_classifier(target[i], source[i], "pair", MODEL)
        elif correct_left[i] != 4 and correct_right[i] == 4:
            correct_left[i] = purity_classifier(target[i], source[i], "left", MODEL)
        elif correct_left[i] == 4 and correct_right[i] != 4:
            correct_right[i] = purity_classifier(target[i], source[i], "right", MODEL)
        else:
            pass

        if correct_left[i] == 1 and correct_right[i] == 1:
            case[i] = 1
        elif correct_left[i] != 1 and correct_right[i] == 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 2
        elif correct_left[i] == 1 and correct_right[i] != 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 2
        elif  correct_left[i] != 1 and  correct_right[i] != 1 and correct_left[i] != 4 and correct_right[i] != 4:
            case[i] = 3
        elif correct_left[i] == 4 or  correct_right[i] == 4:
            case[i] = 4
        
    return case, correct_right, correct_left

In [7]:
NN_predict_2 = full_predictions[NN_n_match_2_idx[0]]
NN_target_2 = full_targets[NN_n_match_2_idx[0]]

In [8]:
NN_predict_1 = full_predictions[NN_n_match_1_idx[0]]
NN_target_1 = full_targets[NN_n_match_1_idx[0]]

In [9]:
chi2_predict_2 = chi2_result[chi2_n_match_2_idx[0]]
chi2_target_2 = delta_R_result[chi2_n_match_2_idx[0]]

chi2_predict_1 = chi2_result[chi2_n_match_1_idx[0]]
chi2_target_1 = delta_R_result[chi2_n_match_1_idx[0]]

In [10]:
NN_case, NN_correct_right, NN_correct_left = compute(full_predictions, full_targets)
NN_case_1, NN_correct_right_1, NN_correct_left_1 = compute(NN_predict_1, NN_target_1)
NN_cas_2, NN_correct_right_2, NN_correct_left_2 = compute(NN_predict_2, NN_target_2)

100%|██████████| 386971/386971 [00:08<00:00, 47306.86it/s]


In [11]:
chi2_case, chi2_correct_right, chi2_correct_left = compute(chi2_result, delta_R_result)
chi2_case_1, chi2_correct_right_1, chi2_correct_left_1 = compute(chi2_predict_1, chi2_target_1)
chi2_cas_2, chi2_correct_right_2, chi2_correct_left_2 = compute(chi2_predict_2, chi2_target_2)

100%|██████████| 387031/387031 [00:08<00:00, 44856.67it/s]


In [12]:
print(f'No filterd event purity and top purity.')

NN_event_purity = np.sum(NN_case==1)/( np.sum(NN_case==1) + np.sum(NN_case==2) + np.sum(NN_case==3))
NN_two_top_purity = ( np.sum(NN_correct_right_2 ==1) + np.sum(NN_correct_left_2==1)  )/( np.sum(NN_correct_right_2 ==1) + np.sum(NN_correct_left_2==1) + np.sum(NN_correct_right_2 ==0) + np.sum(NN_correct_left_2==0) )
NN_single_top_purity = ( np.sum(NN_correct_right_1 == 1) + np.sum(NN_correct_left_1 == 1)  )/(  len(NN_correct_right_1) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity:.3f}')

print("-"*80)

chi2_event_purity = np.sum(chi2_case==1)/( np.sum(chi2_case==1) + np.sum(chi2_case==2) + np.sum(chi2_case==3))
chi2_two_top_purity = ( np.sum(chi2_correct_right_2 ==1) + np.sum(chi2_correct_left_2==1)  )/( np.sum(chi2_correct_right_2 ==1) + np.sum(chi2_correct_left_2==1) + np.sum(chi2_correct_right_2 ==0) + np.sum(chi2_correct_left_2==0) )
chi2_single_top_purity = ( np.sum(chi2_correct_right_1==1) + np.sum(chi2_correct_left_1==1)  )/( len(chi2_correct_right_1) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity:.3f}')

No filterd event purity and top purity.
Event putiry (no filter, NN)               : 0.867
Two top putiry (both matched, NN)          : 0.909
Single top putiry (partial matched, NN)    : 0.497
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.398
Two top putiry (both matched, chi2)        : 0.491
Single top putiry (partial matched, chi2)  : 0.306


In [13]:

NN_n_match_2_six_jet = np.intersect1d(NN_n_match_2_idx, NN_six_jet_idx)
NN_n_match_1_six_jet = np.intersect1d(NN_n_match_1_idx, NN_six_jet_idx)

NN_predict_six_jet = full_predictions[NN_six_jet_idx]
NN_predict_six_jet_2 = full_predictions[NN_n_match_2_six_jet]
NN_predict_six_jet_1 = full_predictions[NN_n_match_1_six_jet]

NN_target_six_jet = full_targets[NN_six_jet_idx]
NN_target_six_jet_2 = full_targets[NN_n_match_2_six_jet]
NN_target_six_jet_1 = full_targets[NN_n_match_1_six_jet]

NN_n_match_2_seven_jet = np.intersect1d(NN_n_match_2_idx, NN_seven_jet_idx)
NN_n_match_1_seven_jet = np.intersect1d(NN_n_match_1_idx, NN_seven_jet_idx)

NN_predict_seven_jet = full_predictions[NN_seven_jet_idx]
NN_predict_seven_jet_2 = full_predictions[NN_n_match_2_seven_jet]
NN_predict_seven_jet_1 = full_predictions[NN_n_match_1_seven_jet]

NN_target_seven_jet = full_targets[NN_seven_jet_idx]
NN_target_seven_jet_2 = full_targets[NN_n_match_2_seven_jet]
NN_target_seven_jet_1 = full_targets[NN_n_match_1_seven_jet]

NN_n_match_2_eight_jet = np.intersect1d(NN_n_match_2_idx, NN_eight_jet_idx)
NN_n_match_1_eight_jet = np.intersect1d(NN_n_match_1_idx, NN_eight_jet_idx)

NN_predict_eight_jet = full_predictions[NN_eight_jet_idx]
NN_predict_eight_jet_2 = full_predictions[NN_n_match_2_eight_jet]
NN_predict_eight_jet_1 = full_predictions[NN_n_match_1_eight_jet]

NN_target_eight_jet = full_targets[NN_eight_jet_idx]
NN_target_eight_jet_2 = full_targets[NN_n_match_2_eight_jet]
NN_target_eight_jet_1 = full_targets[NN_n_match_1_eight_jet]

chi2_n_match_2_six_jet = np.intersect1d(chi2_n_match_2_idx, chi2_six_jet)
chi2_n_match_1_six_jet = np.intersect1d(chi2_n_match_1_idx, chi2_six_jet)

chi2_predict_six_jet = chi2_result[chi2_six_jet]
chi2_predict_six_jet_2 = chi2_result[chi2_n_match_2_six_jet]
chi2_predict_six_jet_1 = chi2_result[chi2_n_match_1_six_jet]

chi2_target_six_jet = delta_R_result[chi2_six_jet]
chi2_target_six_jet_2 = delta_R_result[chi2_n_match_2_six_jet]
chi2_target_six_jet_1 = delta_R_result[chi2_n_match_1_six_jet]

chi2_n_match_2_seven_jet = np.intersect1d(chi2_n_match_2_idx, chi2_seven_jet)
chi2_n_match_1_seven_jet = np.intersect1d(chi2_n_match_1_idx, chi2_seven_jet)

chi2_predict_seven_jet = chi2_result[chi2_seven_jet]
chi2_predict_seven_jet_2 = chi2_result[chi2_n_match_2_seven_jet]
chi2_predict_seven_jet_1 = chi2_result[chi2_n_match_1_seven_jet]

chi2_target_seven_jet = delta_R_result[chi2_seven_jet]
chi2_target_seven_jet_2 = delta_R_result[chi2_n_match_2_seven_jet]
chi2_target_seven_jet_1 = delta_R_result[chi2_n_match_1_seven_jet]

chi2_n_match_2_eight_jet = np.intersect1d(chi2_n_match_2_idx, chi2_eight_jet)
chi2_n_match_1_eight_jet = np.intersect1d(chi2_n_match_1_idx, chi2_eight_jet)

chi2_predict_eight_jet = chi2_result[chi2_eight_jet]
chi2_predict_eight_jet_2 = chi2_result[chi2_n_match_2_eight_jet]
chi2_predict_eight_jet_1 = chi2_result[chi2_n_match_1_eight_jet]

chi2_target_eight_jet = delta_R_result[chi2_eight_jet]
chi2_target_eight_jet_2 = delta_R_result[chi2_n_match_2_eight_jet]
chi2_target_eight_jet_1 = delta_R_result[chi2_n_match_1_eight_jet]

In [14]:
NN_case_six_jet, NN_correct_right_six_jet, NN_correct_left_six_jet = compute(NN_predict_six_jet, NN_target_six_jet)
NN_case_1_six_jet, NN_correct_right_1_six_jet, NN_correct_left_1_six_jet = compute(NN_predict_six_jet_1, NN_target_six_jet_1)
NN_cas_2_six_jet, NN_correct_right_2_six_jet, NN_correct_left_2_six_jet = compute(NN_predict_six_jet_2, NN_target_six_jet_2)

chi2_case_six_jet, chi2_correct_right_six_jet, chi2_correct_left_six_jet = compute(chi2_predict_six_jet, chi2_target_six_jet)
chi2_case_1_six_jet, chi2_correct_right_1_six_jet, chi2_correct_left_1_six_jet = compute(chi2_predict_six_jet_1, chi2_target_six_jet_1)
chi2_cas_2_six_jet, chi2_correct_right_2_six_jet, chi2_correct_left_2_six_jet = compute(chi2_predict_six_jet_2, chi2_target_six_jet_2)

100%|██████████| 88373/88373 [00:01<00:00, 46217.49it/s]


In [15]:
print(f'No filterd event purity and top purity.')

NN_event_purity_six_jet = np.sum(NN_case_six_jet==1)/( np.sum(NN_case_six_jet==1) + np.sum(NN_case_six_jet==2) + np.sum(NN_case_six_jet==3))
NN_two_top_purity_six_jet = ( np.sum(NN_correct_right_2_six_jet ==1) + np.sum(NN_correct_left_2_six_jet==1)  )/( np.sum(NN_correct_right_2_six_jet ==1) + np.sum(NN_correct_left_2_six_jet==1) + np.sum(NN_correct_right_2_six_jet ==0) + np.sum(NN_correct_left_2_six_jet==0) )
NN_single_top_purity_six_jet = ( np.sum(NN_correct_right_1_six_jet == 1) + np.sum(NN_correct_left_1_six_jet == 1)  )/(  len(NN_correct_right_1_six_jet) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_six_jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_six_jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_six_jet:.3f}')

print("-"*80)

chi2_event_purity_six_jet = np.sum(chi2_case_six_jet==1)/( np.sum(chi2_case_six_jet==1) + np.sum(chi2_case_six_jet==2) + np.sum(chi2_case_six_jet==3))
chi2_two_top_purity_six_jet = ( np.sum(chi2_correct_right_2_six_jet ==1) + np.sum(chi2_correct_left_2_six_jet==1)  )/( np.sum(chi2_correct_right_2_six_jet ==1) + np.sum(chi2_correct_left_2_six_jet==1) + np.sum(chi2_correct_right_2_six_jet ==0) + np.sum(chi2_correct_left_2_six_jet==0) )
chi2_single_top_purity_six_jet = ( np.sum(chi2_correct_right_1_six_jet==1) + np.sum(chi2_correct_left_1_six_jet==1)  )/( len(chi2_correct_right_1_six_jet ) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_six_jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_six_jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_six_jet:.3f}')

No filterd event purity and top purity.
Event putiry (no filter, NN)               : 0.930
Two top putiry (both matched, NN)          : 0.948
Single top putiry (partial matched, NN)    : 0.539
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.638
Two top putiry (both matched, chi2)        : 0.668
Single top putiry (partial matched, chi2)  : 0.330


In [16]:
NN_case_seven_jet, NN_correct_right_seven_jet, NN_correct_left_seven_jet = compute(NN_predict_seven_jet, NN_target_seven_jet)
NN_case_1_seven_jet, NN_correct_right_1_seven_jet, NN_correct_left_1_seven_jet = compute(NN_predict_seven_jet_1, NN_target_seven_jet_1)
NN_cas_2_seven_jet, NN_correct_right_2_seven_jet, NN_correct_left_2_seven_jet = compute(NN_predict_seven_jet_2, NN_target_seven_jet_2)

chi2_case_seven_jet, chi2_correct_right_seven_jet, chi2_correct_left_seven_jet = compute(chi2_predict_seven_jet, chi2_target_seven_jet)
chi2_case_1_seven_jet, chi2_correct_right_1_seven_jet, chi2_correct_left_1_seven_jet = compute(chi2_predict_seven_jet_1, chi2_target_seven_jet_1)
chi2_cas_2_seven_jet, chi2_correct_right_2_seven_jet, chi2_correct_left_2_seven_jet = compute(chi2_predict_seven_jet_2, chi2_target_seven_jet_2)

100%|██████████| 126920/126920 [00:02<00:00, 45624.02it/s]


In [17]:
print(f'No filterd event purity and top purity.')

NN_event_purity_seven_jet = np.sum(NN_case_seven_jet==1)/( np.sum(NN_case_seven_jet==1) + np.sum(NN_case_seven_jet==2) + np.sum(NN_case_seven_jet==3))
NN_two_top_purity_seven_jet = ( np.sum(NN_correct_right_2_seven_jet ==1) + np.sum(NN_correct_left_2_seven_jet==1)  )/( np.sum(NN_correct_right_2_seven_jet ==1) + np.sum(NN_correct_left_2_seven_jet==1) + np.sum(NN_correct_right_2_seven_jet ==0) + np.sum(NN_correct_left_2_seven_jet==0) )
NN_single_top_purity_seven_jet = ( np.sum(NN_correct_right_1_seven_jet == 1) + np.sum(NN_correct_left_1_seven_jet == 1)  )/(  len(NN_correct_right_1_seven_jet) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_seven_jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_seven_jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_seven_jet:.3f}')

print("-"*80)

chi2_event_purity_seven_jet = np.sum(chi2_case_seven_jet==1)/( np.sum(chi2_case_seven_jet==1) + np.sum(chi2_case_seven_jet==2) + np.sum(chi2_case_seven_jet==3))
chi2_two_top_purity_seven_jet = ( np.sum(chi2_correct_right_2_seven_jet ==1) + np.sum(chi2_correct_left_2_seven_jet==1)  )/( np.sum(chi2_correct_right_2_seven_jet ==1) + np.sum(chi2_correct_left_2_seven_jet==1) + np.sum(chi2_correct_right_2_seven_jet ==0) + np.sum(chi2_correct_left_2_seven_jet==0) )
chi2_single_top_purity_seven_jet = ( np.sum(chi2_correct_right_1_seven_jet==1) + np.sum(chi2_correct_left_1_seven_jet==1)  )/( len(chi2_correct_right_1_seven_jet ) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_seven_jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_seven_jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_seven_jet:.3f}')

No filterd event purity and top purity.
Event putiry (no filter, NN)               : 0.878
Two top putiry (both matched, NN)          : 0.916
Single top putiry (partial matched, NN)    : 0.504
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.432
Two top putiry (both matched, chi2)        : 0.526
Single top putiry (partial matched, chi2)  : 0.324


In [18]:
NN_case_eight_jet, NN_correct_right_eight_jet, NN_correct_left_eight_jet = compute(NN_predict_eight_jet, NN_target_eight_jet)
NN_case_1_eight_jet, NN_correct_right_1_eight_jet, NN_correct_left_1_eight_jet = compute(NN_predict_eight_jet_1, NN_target_eight_jet_1)
NN_cas_2_eight_jet, NN_correct_right_2_eight_jet, NN_correct_left_2_eight_jet = compute(NN_predict_eight_jet_2, NN_target_eight_jet_2)

chi2_case_eight_jet, chi2_correct_right_eight_jet, chi2_correct_left_eight_jet = compute(chi2_predict_eight_jet, chi2_target_eight_jet)
chi2_case_1_eight_jet, chi2_correct_right_1_eight_jet, chi2_correct_left_1_eight_jet = compute(chi2_predict_eight_jet_1, chi2_target_eight_jet_1)
chi2_cas_2_eight_jet, chi2_correct_right_2_eight_jet, chi2_correct_left_2_eight_jet = compute(chi2_predict_eight_jet_2, chi2_target_eight_jet_2)

100%|██████████| 171738/171738 [00:03<00:00, 44759.78it/s]


In [21]:
print(f'No filterd event purity and top purity.')

NN_event_purity_eight_jet = np.sum(NN_case_eight_jet==1)/( np.sum(NN_case_eight_jet==1) + np.sum(NN_case_eight_jet==2) + np.sum(NN_case_eight_jet==3))
NN_two_top_purity_eight_jet = ( np.sum(NN_correct_right_2_eight_jet ==1) + np.sum(NN_correct_left_2_eight_jet==1)  )/( np.sum(NN_correct_right_2_eight_jet ==1) + np.sum(NN_correct_left_2_eight_jet==1) + np.sum(NN_correct_right_2_eight_jet ==0) + np.sum(NN_correct_left_2_eight_jet==0) )
NN_single_top_purity_eight_jet = ( np.sum(NN_correct_right_1_eight_jet == 1) + np.sum(NN_correct_left_1_eight_jet == 1)  )/(  len(NN_correct_right_1_eight_jet) )

print(f'Event putiry (no filter, NN)               : {NN_event_purity_eight_jet:.3f}')
print(f'Two top putiry (both matched, NN)          : {NN_two_top_purity_eight_jet:.3f}')
print(f'Single top putiry (partial matched, NN)    : {NN_single_top_purity_eight_jet:.3f}')

print("-"*80)

chi2_event_purity_eight_jet = np.sum(chi2_case_eight_jet==1)/( np.sum(chi2_case_eight_jet==1) + np.sum(chi2_case_eight_jet==2) + np.sum(chi2_case_eight_jet==3))
chi2_two_top_purity_eight_jet = ( np.sum(chi2_correct_right_2_eight_jet ==1) + np.sum(chi2_correct_left_2_eight_jet==1)  )/( np.sum(chi2_correct_right_2_eight_jet ==1) + np.sum(chi2_correct_left_2_eight_jet==1) + np.sum(chi2_correct_right_2_eight_jet ==0) + np.sum(chi2_correct_left_2_eight_jet==0) )
chi2_single_top_purity_eight_jet = ( np.sum(chi2_correct_right_1_eight_jet==1) + np.sum(chi2_correct_left_1_eight_jet==1)  )/( len(chi2_correct_right_1_eight_jet ) )
print(f'Event putiry (no filter, chi2)             : {chi2_event_purity_eight_jet:.3f}')
print(f'Two top putiry (both matched, chi2)        : {chi2_two_top_purity_eight_jet:.3f}')
print(f'Single top putiry (partial matched, chi2)  : {chi2_single_top_purity_eight_jet:.3f}')

No filterd event purity and top purity.
Event putiry (no filter, NN)               : 0.826
Two top putiry (both matched, NN)          : 0.883
Single top putiry (partial matched, NN)    : 0.450
--------------------------------------------------------------------------------
Event putiry (no filter, chi2)             : 0.250
Two top putiry (both matched, chi2)        : 0.375
Single top putiry (partial matched, chi2)  : 0.264
